In [2]:
from synthetix_v3.base_perps import TestBasePerps
import polars as pl

In [3]:
base_perp = TestBasePerps()

In [4]:
market_updates_df = base_perp.get_market_updates()

In [5]:
market_updates_df.dtypes

marketUpdateds_timestamp                   int64
markets_id                                 int64
marketUpdateds_price                     float64
marketUpdateds_skew                      float64
marketUpdateds_size                      float64
marketUpdateds_sizeDelta                 float64
marketUpdateds_currentFundingRate        float64
marketUpdateds_currentFundingVelocity    float64
markets_marketSymbol                      object
dtype: object

In [6]:
polars_df = pl.from_pandas(market_updates_df)

In [7]:
polars_df.shape

(2984, 9)

In [8]:
# save to perps_v3_competition folder
polars_df.write_parquet('../perps_v3_competition/market_updates.parquet')

In [9]:
# divide by decimals
polars_df = polars_df.with_columns([
    pl.from_epoch("marketUpdateds_timestamp").alias("datetime"),
    (pl.col('marketUpdateds_price') / 10 ** 18),
    (pl.col('marketUpdateds_skew') / 10 ** 18),
    (pl.col('marketUpdateds_size') / 10 ** 18),
    (pl.col('marketUpdateds_sizeDelta') / 10 ** 18),
    (pl.col('marketUpdateds_currentFundingRate') / 10 ** 18),
    (pl.col('marketUpdateds_currentFundingVelocity') / 10 ** 18)
])

In [10]:
polars_df.groupby('markets_marketSymbol').agg([pl.count()]).sort(by='count', descending=True)

/tmp/ipykernel_12582/813217849.py:1: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  polars_df.groupby('markets_marketSymbol').agg([pl.count()]).sort(by='count', descending=True)


markets_marketSymbol,count
str,u32
"""ETH""",1127
"""SNX""",622
"""BTC""",565
"""OP""",350
"""LINK""",320
